In [4]:
from numpy import loadtxt
import copy
import csv

In [5]:
def edgeGraph_to_listrepGraph(d):
    '''
    This function convert a file containing all edges of a directed graph
    to its list representation form
    
    input:
            d is the list contain all rows of the file
    ourput:
            G is a dictionary as list representation of the graph. keys are source vortecx
            and values (in form list) have the destionations 
    '''
    i = 1
    m = len(d) # number of edges (number of rows in the file)
    G = {}
    G['1'] = []
    x = 0
    node_num_max = 0 # keep track of maximum node number. The graph dictionary should include all of them even with empty list
    
    while x < m:
        if int(d[x][0]) == i:
            G[str(i)].append(int(d[x][1]))
            if i > node_num_max:
                node_num_max = i
            if int(d[x][1]) > node_num_max:
                node_num_max = int(d[x][1])
            x = x + 1
            p = 1
        else:
            i += 1
            G[str(i)] = []
            if i > node_num_max:
                node_num_max = i
            if int(d[x][1]) > node_num_max:
                node_num_max = int(d[x][1])
                
    while i < node_num_max:
        i += 1
        G[str(i)] = []
    return G    

In [6]:
def DFS(G,i):
    '''
    Deep-First Search algortihm
    
    Input:
            G is the input graph
            i is the node that algorithm finds all reaschable vortecies from it
    Outputs:
            leader is a dictionary represnting each node with values as leaders
            f is a dictionary represnting each node with values as finishing time
            G_explored is a dictionary represnting each node with values (T/F) as explored/unexplored
    '''
    global t, s, G_explored, leader, f
    
    G_explored[str(i)] = True # make i explored
    leader[str(i)] = str(s)
    for j in G[str(i)]:
        if G_explored[str(j)] == False:
            DFS(G,j)
    t += 1
    f[str(i)] = t
    return leader, f, G_explored

In [7]:
def DFS_LOOP(G):
    '''
    DFS-LOOP algorithm for SSC finding
    
    Inputs:
            G is the input graph 
    Outputs:
            leader is a dictionary represnting each node with values as leaders
            f is a dictionary represnting each node with values as finishing time
            G_explored is a dictionary represnting each node with values (T/F) as explored/unexplored
    '''
    global t, s, G_explored, leader, f
    for i in range(n,0,-1):
        if G_explored[str(i)] == False:
            s = i
            leader, f, G_explored = DFS(G,i)
    return leader, f, G_explored

In [16]:
# reading the input file and outputing dG as a list containing all rows
with open('Test_01.txt') as f:
    reader = csv.reader(f, delimiter=" ")
    dG = list(reader)
dG.sort(key = lambda x: int(x[0])) # sort based on first column
    
# creating dGrev, switching first and second column of dG and sort based on first column
X = dG
X =  dG.copy()
X.sort(key = lambda x: int(x[1])) 
dGrev = []
k = 0
for i in X:
    dGrev.append([i[1]])
    dGrev[k].append(i[0])
    k += 1

In [17]:
# creating list represntation of G and Grev (inverse of G) 
G = edgeGraph_to_listrepGraph(dG)
print(G)
Grev = edgeGraph_to_listrepGraph(dGrev)
print(Grev)

{'1': [4], '2': [8], '3': [6], '4': [7], '5': [2], '6': [9], '7': [1], '8': [6, 5], '9': [7, 3]}
{'1': [7], '2': [5], '3': [9], '4': [1], '5': [8], '6': [3, 8], '7': [4, 9], '8': [2], '9': [6]}


In [21]:
'''Doing SSC determinations'''
# initializing Global variables
G_explored = {x: False for x in G} # intially all nodes are unexplored in G
leader = copy.deepcopy(G) # intilize leader dictionary
f = copy.deepcopy(G) # finishing time dictionary
t = 0
s = 0
n = len(G)
# 1 - Running DFS_LOOP on Grev
Grev_leader, Grev_f, Grev_explored = DFS_LOOP(Grev)

# 2 - Updating node labels of G with finishing time obtained from Grev
# updating keys 
G_key_updated = dict((str(Grev_f[key]), value) for (key, value) in G.items())
# updating values 
G_full_updated= {}
for (key, value) in G_key_updated.items():
    G_full_updated[str(key)] = []
    for i in range(0,len(G_key_updated[str(key)])):
        G_full_updated[str(key)].append(Grev_f[str(G_key_updated[str(key)][i])])


# 3 - Running DFS_LOOP on G with updated node lables
G_explored = {x: False for x in G} # intially all nodes are unexplored in G
leader = copy.deepcopy(G) # intilize leader dictionary
f = copy.deepcopy(G) # finishing time dictionary
t = 0
s = 0
n = len(G)
G_leader, G_f, G_explored = DFS_LOOP(G_full_updated)

'''SSC sizes analysis'''
from collections import Counter
# counting duplicate values in G_leader
# source: https://stackoverflow.com/questions/52027616/how-to-count-the-same-values-in-a-dict
SSC_sizes = Counter(G_leader.values())
print('SSC summary: leading nodes and number of nodes inside each SCC')
print(SSC_sizes)
# sort SSC_sizes (counted) keys based on maximum duplication (only 5 first items), represtens size of each SSC
# source: https://stackoverflow.com/questions/7197315/5-maximum-values-in-a-python-dictionary
sorted_based_on_val = sorted(SSC_sizes, key=SSC_sizes.get, reverse=True)
print('\nLeading keys sorted by their SCC sizes')
print(sorted_based_on_val)
# finding values of sorted keys, as bigest SSC sizes
maximum_size_of_SSC = [SSC_sizes[str(i)] for i in sorted_based_on_val]
print('\nSorted SSC sizes')
print(maximum_size_of_SSC)

SSC summary: leading nodes and number of nodes inside each SCC
Counter({'6': 3, '4': 3, '9': 3})

Leading keys sorted by their SCC sizes
['6', '4', '9']

Sorted SSC sizes
[3, 3, 3]
